The following notebook was auto-generated by GitHub Copilot Chat and is meant for initial setup only

# Introduction to Prompt Engineering
Prompt engineering is the process of designing and optimizing prompts for natural language processing tasks. It involves selecting the right prompts, tuning their parameters, and evaluating their performance. Prompt engineering is crucial for achieving high accuracy and efficiency in NLP models. In this section, we will explore the basics of prompt engineering using the OpenAI models for exploration.

### Exercise 1: Tokenization
Explore Tokenization using tiktoken, an open-source fast tokenizer from OpenAI
See [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst) for more examples.


In [2]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
This is my new prompt. \
I wrote it all by myself. \
Are you proud of me? \
"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

[198, 2028, 374, 856, 502, 10137, 13, 358, 6267, 433, 682, 555, 7182, 13, 8886, 499, 12691, 315, 757, 30, 220]


[b'\n',
 b'This',
 b' is',
 b' my',
 b' new',
 b' prompt',
 b'.',
 b' I',
 b' wrote',
 b' it',
 b' all',
 b' by',
 b' myself',
 b'.',
 b' Are',
 b' you',
 b' proud',
 b' of',
 b' me',
 b'?',
 b' ']

### Exercise 2: Validate OpenAI API Key Setup

Run the code below to verify that your OpenAI endpoint is set up correctly. The code just tries a simple basic prompt and validates the completion. Input `oh say can you see` should complete along the lines of `by the dawn's early light..`


In [1]:
# The OpenAI SDK was updated on Nov 8, 2023 with new guidance for migration
# See: https://github.com/openai/openai-python/discussions/742

## Updated
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']

## Updated
def get_completion(prompt):
    messages = [{"role": "user", "content": prompt}]       
    response = client.chat.completions.create(   
        model=deployment,                                         
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens=1024
    )
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
oh say can you see
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt)
print(response)

By the dawn's early light,


### Exercise 3: Fabrications
Explore what happens when you ask the LLM to return completions for a prompt about a topic that may not exist, or about topics that it may not know about because it was outside it's pre-trained dataset (more recent). See how the response changes if you try a different prompt, or a different model.

In [5]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt)
print(response)

Title: The Martian War of 2076

Grade Level: High School

Objective: Students will be able to understand the causes, events, and consequences of the Martian War of 2076.

Materials:
- Textbook or online resources on the Martian War of 2076
- Maps of Mars and Earth
- Images and videos of Martian colonies and military equipment
- Writing materials

Procedure:

Introduction (10 minutes)
- Begin by asking students if they have heard of the Martian War of 2076 and what they know about it.
- Explain that the Martian War of 2076 was a conflict between Earth and Mars that lasted for several years and had significant consequences for both planets.

Causes of the War (20 minutes)
- Provide students with background information on the causes of the Martian War of 2076, including the political, economic, and environmental factors that led to the conflict.
- Ask students to discuss in small groups the causes of the war and how they might have been prevented.

Events of the War (30 minutes)
- Using m

### Exercise 4: Instruction Based 
Use the "text" variable to set the primary content 
and the "prompt" variable to provide an instruction related to that primary content.

Here we ask the model to summarize the text for a second-grade student

In [6]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Tatooine was a sparsely inhabited circumbinary desert planet located in the galaxy's Outer Rim Territories. Part of a binary star system, the planet orbited two scorching suns, resulting in the world lacking the necessary surface water to sustain large populations. As a result, many residents of the planet instead drew water from the atmosphere via moisture farms. The planet also had little surface vegetation. It was the homeworld to the native Jawa and Tusken Raider species and of Anakin and Luke Skywalker, who would go on to shape galactic history.

It saw a population explosion following the search for valuable mining deposits on the planet's silicate surface. Finding the ore to have undesirable metallurgical properties, mining firms left the planet en masse, leaving behind valuable equipment, most notably large sandcrawlers, which were later used by the Jawa population. The planet then became a haven for smugglers and criminal activity, and fell under the influence of the Hutt Clan while scattered, isolated settlements of hard-working inhabitants endured. 
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt)
print(response)

Tatooine is a desert planet in a faraway place called the Outer Rim Territories. It has two suns and not a lot of water, so people have to get water from the air. There aren't many plants on the planet either. Some creatures called Jawas and Tusken Raiders live there, and two important people named Anakin and Luke Skywalker are from there too. People used to mine for valuable things on the planet, but they didn't find what they were looking for and left their equipment behind. Then, bad people started doing bad things on the planet, but some good people still live there.


### Exercise 5: Complex Prompt 
Try a request that has system, user and assistant messages 
System sets assistant context
User & Assistant messages provide multi-turn conversation context

Note how the assistant personality is set to "sarcastic" in the system context. 
Try using a different personality context. Or try a different series of input/output messages

In [12]:
response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "What's the meaning of life?"}
    ]
)

print(response.choices[0].message.content)

Ah, the age-old question. Are you sure you want my sarcastic answer or actually expect a serious one? Either way, here it goes: The meaning of life is to struggle through traffic, pay bills, and try to find moments of joy amidst the mundane. Or, you can make your own meaning and do whatever you damn well please. Your call.


### Exercise: Explore Your Intuition
The above examples give you patterns that you can use to create new prompts (simple, complex, instruction etc.) - try creating other exercises to explore some of the other ideas we've talked about like examples, cues and more.